<a href="https://colab.research.google.com/github/amun83/Koscom-Financial-Data-Bootcamp/blob/master/FD104_Text_Mining_OilPrice_com_News_Articles_Step_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. News Article Scraping: https://oilprice.com/

In [0]:
ON_COLAB  = True
DATA_PATH = ''

In [0]:
!pip install -q --upgrade pandas_datareader
!pip install -q --upgrade finance-datareader
!pip install -q --upgrade html5lib
!pip install -q --upgrade lxml
!pip install -q --upgrade seaborn
!pip install -q --upgrade gspread

In [0]:
# utilities
import datetime
import re
import csv
from IPython.display import display

# web scraping
import requests
import urllib.parse
from bs4 import BeautifulSoup as bs
import pandas_datareader as pdr
import FinanceDataReader as fdr

# Math/Stat
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
#import math

# Machine Learning
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

# visualization
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
if ON_COLAB :
  from google.colab import drive
  drive.mount('/gdrive')
  DATA_PATH = '/gdrive/My Drive/crawling/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
pattern = re.compile(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d\d),\s(\d{4})')
# Example: By Julianne Geiger - Dec 14, 2018, 8:00 PM CST

## 함수: 페이지의 News URL 가져오기
 * https://oilprice.com/Latest-Energy-News/World-News

In [0]:
def get_news_urls_by_page(page_num):
  try:
    get_news_urls_by_page.counter += 1
  except:
    get_news_urls_by_page.counter = 0
    
  page_url = 'https://oilprice.com/Latest-Energy-News/World-News/Page-' + str(page_num) + '.html'
  # https://oilprice.com/Latest-Energy-News/World-News/Page-865.html

  try:
    r = requests.get(page_url)
    if get_news_urls_by_page.counter % 10 == 0:
      print('Request(GET) >> OK: ' + page_url)
  except:
    print('Request(GET) >> Failed: ' + page_url)
    return None

  soup = bs(r.text)
  items = soup.find_all('div', {'class':'categoryArticle__content'})

  if not items:
    print('Request(GET) >> Empty Page: ' + page_url)
    return None
  
  return [item.find('a', href = True)['href'] for item in items]

## 함수: News 내용 가져오기 (작성일, 제목, 내용, 작성자)
 * https://oilprice.com/Latest-Energy-News/World-News/Surprise-Inventory-Build-Sends-Oil-Prices-Even-Lower.html

In [0]:
def get_news_content(url):
  try:
    get_news_content.counter += 1
  except:
    get_news_content.counter = 0
  
  try:
    r = requests.get(url)
    if get_news_content.counter % 100 == 0:
      print('[' + str(get_news_content.counter) + '] Request(GET) >> OK: ' + url)
  except:
    print('[' + str(get_news_content.counter) + '] Request(GET) >> Failed: ' + url)
    return None

  soup = bs(r.text)

  title = soup.find('h1').text.strip()

  date = soup.find('span', {'class':'article_byline'}).text

  match = re.search(pattern, date)
  mmm, dd, yyyy = match.groups(0)
  date = yyyy + "-" + mmm + "-" + dd

  raw_content = soup.find('div', {'id':'news-content'}).find_all('p')
  content = ' '.join([p.text.strip() for p in raw_content])
  
  try:
    author = soup.find('p', {'class':'authorBox__authorName'}).text.strip()
  except:
    author = ''
  
  return date, title, content, author

## 페이지별 News URL 긁어오기

In [28]:
news_urls =[]

for i in range(1, 1000):
  urls = get_news_urls_by_page(i)
  if not urls:
    break
    
  news_urls += urls

Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-1.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-11.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-21.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-31.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-41.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-51.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-61.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-71.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-81.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-91.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-101.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-

In [29]:
print(len(news_urls))
print(news_urls[-1])

8659
https://oilprice.com/Latest-Energy-News/World-News/Romania-Posts-2011-2035-National-Energy-Strategy-Online.html


In [0]:
with open(DATA_PATH + 'oilprice.com_news_urls.csv', 'w') as f:
  fw = csv.writer(f)
  fw.writerows([[url] for url in news_urls])

In [31]:
'''
with open(DATA_PATH + 'oilprice.com_news_urls.csv', 'r') as f:
  fr = csv.reader(f)
  news_urls = [u for u in fr]
  
print(len(news_urls))
print(news_urls[-1])
'''

"\nwith open(DATA_PATH + 'oilprice.com_news_urls.csv', 'r') as f:\n  fr = csv.reader(f)\n  news_urls = [u for u in fr]\n  \nprint(len(news_urls))\nprint(news_urls[-1])\n"

## 수집한 URL에서 News 내용 긁어오기

In [35]:
news = [get_news_content(url) + (url.split('/')[-1],) for url in news_urls]
# colums: date, title, content, author, url

[0] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Surprise-Inventory-Build-Sends-Oil-Prices-Even-Lower.html
[100] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Alberta-Forced-To-Buy-Oil-Trains.html
[200] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Plains-All-American-Expands-Permian-Pipeline-Network.html
[300] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Europe-Fuel-Oil-Market-Rocked-As-US-Threatens-To-Slow-Sulfur-Cap-Rollout.html
[400] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Saudi-Aramco-Set-To-Boost-Oil-Capacity-By-550000-Bpd-In-Q4.html
[500] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Baked-Musk-Hints-At-New-Tesla-Smart-Home-Product.html
[600] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Chevron-Starts-Work-On-Iraq-Field-Development-Deal.html
[700] Request(GET) >> OK: https://oilprice.com/Latest-Ene

In [36]:
print(len(news))
display(news[-5:-1])

8659


[('2011-Jun-15',
  'Rosneft Confident Oil Pricing Dispute With China Resolved Soon',
  'China and Rosneft are disputing the price that the Russian company is to provide oil through its Eastern Siberia-Pacific Ocean (ESPO) pipeline, but Rosneft remains optimistic that the issue will shortly be resolved. Rosneft issued a statement observing, "The contract price formula is unchanged. The Chinese side has demonstrated significant progress in the resolution of issues of payment for the supplies, and has paid a significant portion of the debt under the contract. It is these circumstances that instill Rosneft with the confidence that the situation will finally be resolved soon." The ESPO pipeline represents a major strategic shift for Russia, sending its crude oil eastwards to Asian markets rather than the pattern developed in soviet times of shifting energy westwards, first for the indigenous Soviet market and then onwards to Europe. In 2003 former Yukos chairman Mikhail Khodorovskii was cas

In [0]:
with open(DATA_PATH + 'oilprice.com_news.csv', 'w') as f:
  fw = csv.writer(f)
  fw.writerows(news)

## 수집한 데이터는 Google Drive에...
 * https://drive.google.com/drive/folders/16J3gm9rs-zdW2d_h2G0Ctw0qmtFM2oqG?usp=sharing